# Benchmarks - Preprocess DB

**Main considerations when implementing Preprocess DB**

Magma and MagmaClust both work on unaligned sequences of varying sizes.
Yet, we want to be able to perform as many operations in a vectorisable and jittable way.

This mean that we need to pad the sequences to the same length, aligning them in the process, and then mask the padded values.
A mask should be computed at the first step of the process to not be re-computed every time we need to mask the values.

This way, each operation can have custom logic to handle padded values, all while preserving vectorisation and jitting.


---
## Setup

In [177]:
# Standard library
import os

# Config
# os.environ['JAX_ENABLE_X64'] = "True"

In [178]:
# Third party
import jax
from jax import vmap, jit
import jax.numpy as jnp

import numpy as np
import pandas as pd

# Initialize random key
key = jax.random.PRNGKey(41)

In [179]:
# Local


In [180]:
# Set constants
M = 500  # Number of sequences
MIN_N = 50  # Minimum sequence length
MAX_N = 100  # Maximum sequence length
grid = jnp.arange(-500., 500., dtype=jnp.float32)  # Grid to pick inputs from

In [181]:
"""
# Smaller constants for testing
M = 4  # Number of sequences
MIN_N = 2  # Minimum sequence length
MAX_N = 5  # Maximum sequence length
grid = jnp.arange(5, 10, 1, dtype=jnp.float64)  # Grid to pick inputs from
"""

'\n# Smaller constants for testing\nM = 4  # Number of sequences\nMIN_N = 2  # Minimum sequence length\nMAX_N = 5  # Maximum sequence length\ngrid = jnp.arange(5, 10, 1, dtype=jnp.float64)  # Grid to pick inputs from\n'

---
## Data

In [182]:
def generate_dummy_db(M: int, MIN_N: int, MAX_N: int, grid: jnp.array, key: jnp.array):
	# We fill DB with random sequences
	data = []
	for m in range(M):
		key, subkey = jax.random.split(key)
		n_points = jax.random.randint(subkey, (), MIN_N, MAX_N)
		inputs = jax.random.choice(subkey, grid, (n_points,), replace=False)
		for n, i in zip(range(n_points), inputs):
			key, subkey1, subkey2 = jax.random.split(key, 3)
			data.append({
				"ID": m,
				"Input": i.item(),
				"Output": jax.random.uniform(subkey2, (), jnp.float32, -5, 5).item()
			})
	return pd.DataFrame(data)

In [183]:
db = generate_dummy_db(M, MIN_N, MAX_N, grid, key)
db

,ID,Input,Output
0,0,-142.0,0.144109
1,0,-173.0,0.106797
2,0,-113.0,1.152955
3,0,-441.0,-3.174350
4,0,-375.0,1.230059
...,...,...,...
37056,499,-136.0,1.238136
37057,499,394.0,1.993368
37058,499,289.0,1.425520
37059,499,-182.0,-1.829005


---
## Current implementation

In [184]:
# Uses .values for even faster iteration
@jit
def extract_id_data(_id, values, all_inputs):
	"""
	Extract data for a given ID from the values array and return a row of padded inputs, padded outputs and mask.

	:param _id:
	:param id_index:
	:param values:
	:param all_inputs:
	:return:
	"""
	padded_input = jnp.full((len(all_inputs),), jnp.nan)
	padded_output = jnp.full((len(all_inputs),), jnp.nan)
	mask = jnp.zeros((len(all_inputs),), dtype=bool)

	idx = jnp.searchsorted(all_inputs, jnp.where(values[:, 0] == _id, values[:, 1], jnp.nan))

	return padded_input.at[idx].set(values[:, 1]), padded_output.at[idx].set(values[:, 2]), mask.at[idx].set(True)


def preprocess_db(db: pd.DataFrame):
	"""

	:param db: the db to process, with columns "ID", "Input" and "Output", in that order
	:return: a tuple of (all_inputs, padded_inputs, padded_outputs, masks)
	   - all_inputs: a matrix of shape (P, ) with all distinct inputs
	   - padded_inputs: a matrix of shape (M, P) where M is the number of sequences and P is the number of distinct
	   inputs. Missing inputs for each sequence are represented as NaNs.
	   - padded_outputs: a matrix of shape (M, P) with corresponding output for each input and NaNs for missing inputs
	   - masks: a matrix of shape (M, P) with 1 where the input is valid and 0 where it is padded
	"""
	# Get all distinct inputs
	all_ids = jnp.array(db["ID"].unique())
	all_inputs = jnp.sort(jnp.array(db["Input"].unique()))

	# Initialise padded inputs, padded outputs and masks
	padded_inputs, padded_outputs, masks = vmap(extract_id_data, in_axes=(0, None, None))(all_ids, db[
		["ID", "Input", "Output"]].values, all_inputs)

	return all_inputs, padded_inputs, padded_outputs, masks

---
## Custom implementation(s)

Defaults we wish to fix:

For M individuals, each having between MIN_N and MAX_N observations over a grid of size G, we currently pad the inputs to the size of the union of all distinct inputs, named P. With a big M, P is likely to approach G, even though MAX_N is much smaller than G.

Later in the process, this leads to inversion of huge padded matrices. As those matrices are padded with identity vectors, the inversion in itself is not made terribly worse. However, the memory footprint is much larger than it needs to be, and the movement of data in and out of the GPU is much more costly.

Rather than "aligning" the inputs to the union of distinct inputs, we can just compute the indices of the inputs in the grid, and use those indices to map individual inputs/covariance matrices/precision matrices to the grid when needed. By filling indices whith nans for smaller sequences, we ensure they can still be processed in a vectorised way, while limiting the memory footprint to it's theoretical minimum.

In this approach, we don't carry a mask arround, but only a mapping of indices to inputs in the distinct inputs grid. This mapping is used to compute the covariance matrices.

In [185]:
@jit
def extract_id_data_new(_id, values, all_inputs, to_fill):
	"""
	Extract data for a given ID from the values array and return a row of padded inputs, padded outputs and index_mappings.

	:param _id:
	:param id_index:
	:param values:
	:param all_inputs:
	:return:
	"""
	inputs_i = jnp.where(values[:,0] == _id, values[:,1], jnp.nan)
	outputs_i = jnp.where(values[:,0] == _id, values[:,2], jnp.nan)
	mappings_i = jnp.searchsorted(all_inputs, inputs_i)

	# Compute index among the whole dataset
	idx_i = jnp.where(jnp.isnan(inputs_i), to_fill.shape[0] + 1, jnp.cumsum(~jnp.isnan(inputs_i)) - 1)

	# Create padded inputs and outputs
	padded_input = jnp.full(to_fill.shape[0], jnp.nan).at[idx_i].set(inputs_i)
	padded_output = jnp.full(to_fill.shape[0], jnp.nan).at[idx_i].set(outputs_i)
	index_mappings = jnp.full(to_fill.shape[0], all_inputs.shape[0] + 1).at[idx_i].set(mappings_i).astype(int)

	return padded_input, padded_output, index_mappings


def preprocess_db_new(db: pd.DataFrame):
	"""

	:param db: the db to process, with columns "ID", "Input" and "Output", in that order
	:return: a tuple of (all_inputs, padded_inputs, padded_outputs, masks)
	   - all_inputs: a matrix of shape (P, ) with all distinct inputs
	   - padded_inputs: a matrix of shape (M, MAX_N) where M is the number of sequences and MAX_N is the max number of points among all sequences. Missing inputs for each sequence are represented as NaNs.
	   - padded_outputs: a matrix of shape (M, MAX_N) with corresponding output for each input and NaNs for missing inputs
	   - index_mappings: a matrix of shape (M, MAX_N) with indices of the inputs in the all_inputs array. Missing inputs for each sequence are represented as -1.
	"""
	# Get all distinct inputs
	db_sorted = db.sort_values(['ID', 'Input'])
	all_ids = jnp.array(db_sorted["ID"].unique())
	all_inputs = jnp.sort(jnp.array(db_sorted["Input"].unique()))
	MAX_N = db_sorted.groupby("ID")["Input"].count().max()  # Maximum number of points in a sequence
	to_fill = jnp.full((MAX_N), jnp.nan)  # Placeholder for padded inputs and outputs

	# Initialise padded inputs, padded outputs and masks
	padded_inputs, padded_outputs, index_mappings = vmap(extract_id_data_new, in_axes=(0, None, None, None))(all_ids,
																											 db_sorted[
																												 ["ID",
																												  "Input",
																												  "Output"]].values,
																											 all_inputs,
																											 to_fill)

	return all_inputs, padded_inputs, padded_outputs, index_mappings

---
## Comparison

In [186]:
all_inputs, padded_inputs, padded_outputs, masks = preprocess_db(db)
all_inputs.shape, padded_inputs.shape, padded_outputs.shape, masks.shape

((1000,), (500, 1000), (500, 1000), (500, 1000))

In [187]:
all_inputs_new, padded_inputs_new, padded_outputs_new, index_mappings_new = preprocess_db_new(db)
all_inputs_new.shape, padded_inputs_new.shape, padded_outputs_new.shape, index_mappings_new.shape

((1000,), (500, 99), (500, 99), (500, 99))

In [188]:
print(f"{len(all_inputs)} distinct inputs, covering {len(all_inputs) / len(grid) * 100:.2f}% of the grid")

1000 distinct inputs, covering 100.00% of the grid


In [189]:
#jnp.allclose(all_inputs, all_inputs_new), jnp.allclose(padded_inputs, padded_inputs_new, equal_nan=True), jnp.allclose(padded_outputs, padded_outputs_new, equal_nan=True), jnp.allclose(masks, masks_new)

## Performance comparison on dummy tasks

### Covariance matrix computation

In [190]:
from MagmaClustPy.kernels import RBFKernel

kern = RBFKernel(length_scale=jnp.array(0.3), variance=jnp.array(1.))

In [191]:
padded_inputs[0], padded_inputs_new[0], index_mappings_new[0]

(Array([  nan,   nan,   nan,   nan, -496.,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan, -468.,   nan,   nan,   nan,
          nan,   nan, -462.,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan, -441.,   nan,   nan,   nan,
          nan,   nan, -435.,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan, -417., -416.,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan, -397.,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan, -375.,
      

In [192]:
@jit
def map_to_full_cov(dense_cov, all_inputs, mapping):
	# return jnp.full((len(all_inputs), len(all_inputs)), jnp.nan).at[tuple(jnp.meshgrid(mapping, mapping))].set(dense_cov)
	return jnp.full((len(all_inputs), len(all_inputs)), jnp.nan).at[jnp.ix_(mapping, mapping)].set(dense_cov)

In [193]:
@jit
def map_to_full_batch(dense_covs, all_inputs, mappings):
    return vmap(map_to_full_cov, in_axes=(0, None, 0))(dense_covs, all_inputs, mappings)

In [194]:
# On single input
np.asarray(kern(padded_inputs[0]))

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan,  1., nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [195]:
np.asarray(map_to_full_cov(kern(padded_inputs_new[0]), all_inputs, index_mappings_new[0]))

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan,  1., nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [196]:
jnp.allclose(kern(padded_inputs[0]), map_to_full_cov(kern(padded_inputs_new[0]), all_inputs, index_mappings_new[0]), equal_nan=True)

Array(True, dtype=bool)

In [197]:
%timeit kern(padded_inputs[0]).block_until_ready()

276 μs ± 12.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [198]:
%timeit map_to_full_cov(kern(padded_inputs_new[0]), all_inputs, index_mappings_new[0]).block_until_ready()

361 μs ± 13 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [199]:
a = kern(padded_inputs).block_until_ready()

In [200]:
b = map_to_full_batch(kern(padded_inputs_new), all_inputs, index_mappings_new).block_until_ready()

In [201]:
jnp.allclose(a, b, equal_nan=True)

Array(True, dtype=bool)

In [202]:
%timeit kern(padded_inputs).block_until_ready()

92.9 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [203]:
%timeit map_to_full_batch(kern(padded_inputs_new), all_inputs, index_mappings_new).block_until_ready()

160 ms ± 5.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Sum of invs

In [204]:
from jax.scipy.linalg import cho_factor, cho_solve

In [205]:
@jit
def full_pad_sum_of_inv(inputs, masks, all_inputs, kernel):
	"""
	compute the sum of inverses of all cross-covariance matrices for each input in inputs.
	It uses a full padding approach, where the inputs are padded to the size of all_inputs and aligned.
	masks gives indices where the inputs are valid (True) or padded (False).

	:param inputs:
	:param masks:
	:param all_inputs:
	:param kernel:
	:return:
	"""
	nugget = 1e-8  # Small value to ensure numerical stability
	covs = kernel(inputs)

	small_eye = jnp.broadcast_to(jnp.eye(covs.shape[-1]), covs.shape)

	# covs is padded with NaNs. Replace them by their corresponding identity rows/cols
	masks_2D = masks[:, :, None] & masks[:, None, :]
	covs = jnp.where(masks_2D, covs, small_eye)

	covs_U, _ = cho_factor(covs + small_eye * nugget)
	covs_inv = cho_solve((covs_U, False), small_eye)
	covs_inv -= jnp.where(masks_2D, 0, small_eye)  # Correction on the diagonal
	return covs_inv.sum(axis=0)

In [206]:
@jit
def dense_pad_sum_of_inv(inputs, mappings, all_inputs, kernel):
	"""
	compute the sum of inverses of all cross-covariance matrices for each input in inputs.
	It uses a dense padding approach, where the inputs are padded to the size of MAX_N and not aligned. Their positions in the all_inputs array are given by mappings.

	:param inputs:
	:param mappings:
	:param all_inputs:
	:param kernel:
	:return:
	"""
	nugget = 1e-8  # Small value to ensure numerical stability
	covs = kernel(inputs)

	small_eye = jnp.broadcast_to(jnp.eye(covs.shape[-1]), covs.shape)

	# Some covs may still end with a few NaNs, so we replace them by their corresponding identity rows/cols
	eyed_covs = jnp.where(jnp.isnan(covs), small_eye, covs)

	covs_U, _ = cho_factor(eyed_covs + small_eye * nugget)
	covs_inv = cho_solve((covs_U, False), small_eye)
	covs_inv -= jnp.where(jnp.isnan(covs), small_eye, 0)  # Correction on the diagonal

	# Now we need to map the covs_inv to the all_inputs array
	return jnp.nan_to_num(map_to_full_batch(covs_inv, all_inputs, mappings)).sum(axis=0)

In [207]:
a = full_pad_sum_of_inv(padded_inputs, masks, all_inputs, kern)

In [208]:
b = dense_pad_sum_of_inv(padded_inputs_new, index_mappings_new, all_inputs, kern)

In [213]:
jnp.allclose(a, b, equal_nan=True, atol=1e-6)

Array(True, dtype=bool)

In [210]:
%timeit full_pad_sum_of_inv(padded_inputs, masks, all_inputs, kern).block_until_ready()

4.28 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [211]:
%timeit dense_pad_sum_of_inv(padded_inputs_new, index_mappings_new, all_inputs, kern).block_until_ready()

256 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
